In [1]:
import os

In [2]:
pwd

'c:\\projects\\Chest-Cancer-Classification-using-MLFlow-and-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\projects\\Chest-Cancer-Classification-using-MLFlow-and-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath: Path = CONFIG_FILE_PATH,
        params_filepath: Path = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([Path(self.config.artifacts_root)])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [8]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [17]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        create_directories([self.config.root_dir])

    def download_file(self) -> str:
        '''
        Fetch data from the source URL to local zip file
        '''
        try:
            # We use the raw ID for better reliability with gdown
            file_id = "1ZyETALkUeNhVp1h8ex0saR8C-zFY5Xjg"
            zip_download_dir = self.config.local_data_file
            
            logger.info(f"Downloading file ID: {file_id} to: {zip_download_dir}")

            if not os.path.exists(zip_download_dir):
                gdown.download(
                    id=file_id, # Using 'id' instead of 'url' is more stable
                    output=zip_download_dir, 
                    quiet=False,
                    fuzzy=True
                )
                logger.info(f"Downloaded data to {zip_download_dir}")
            else:
                logger.info(f"File already exists at {zip_download_dir}. Skipping download.")

        except Exception as e:
            raise e

    def extract_zip_file(self) -> None:
        '''
        Extract zip file to the unzip directory
        '''
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [19]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(config=data_ingestion_config)
    # data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2026-01-18 18:36:05,683: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-01-18 18:36:05,691: INFO: common: yaml file: params.yaml loaded successfully]
[2026-01-18 18:36:05,695: INFO: common: created directory at: artifacts]
[2026-01-18 18:36:05,699: INFO: common: created directory at: artifacts\data_ingestion]
[2026-01-18 18:36:05,709: INFO: common: created directory at: artifacts\data_ingestion]
